## 06-02-02. LLM Science Exam

- 대회: [Kaggle - LLM Science Exam](https://kaggle.com/competitions/kaggle-llm-science-exam) (2023)
  - 과학 5지선다 문제를 LLM으로 풀기
  - 오픈북 방식: 위키피디아를 임베딩/인덱싱 -> 검색 -> RAG로 LLM에 컨텍스트 제공
- 핵심 변경점: RAPIDS로 GPU 가속
  - `pandas` -> `cuDF` (GPU DataFrame)
  - `FAISS` -> `cuVS CAGRA` (GPU ANN 검색)
- 실행 시간: **10분 20초 -> 3분 20초** (약 3배 단축)

#### 전체 파이프라인

1. 라이브러리 임포트 및 환경 설정
2. 텍스트 문장 분할 함수 정의
3. 데이터 로딩 (위키피디아, 훈련 데이터)
4. 프롬프트 임베딩 생성 -> FAISS로 관련 문서 검색
5. 검색된 문서의 텍스트 로딩
6. 문서를 문장 단위로 분할 -> 임베딩 생성
7. 질문 + 선택지 결합 -> 임베딩 생성
8. **cuVS CAGRA**로 GPU 유사도 검색 -> 컨텍스트 추출
9. 결과 저장 및 확인

### ① 라이브러리 임포트 및 환경 설정

- `cudf as pd`: pandas 대신 **cuDF**로 GPU 가속 DataFrame 처리
- `cuvs.neighbors.cagra`: GPU 기반 ANN(근사 최근접 이웃) 검색 알고리즘
- `SentenceTransformer`: 문장을 고정 크기 벡터(임베딩)로 변환
- `blingfire`: 빠른 문장 분할(sentence splitting) 라이브러리
- `faiss`: CPU 기반 벡터 인덱스 (1차 문서 검색에 사용)
- 설정값
  - `MAX_LENGTH = 384`: 임베딩 시 최대 토큰 수
  - `BATCH_SIZE = 16`: 배치당 처리할 문장 수
  - `NUM_SENTENCES_INCLUDE = 3`: 컨텍스트에 포함할 유사 문장 개수

In [ ]:
!pip install -q faiss-cpu
!pip install -q -U sentence-transformers
!pip install -q --upgrade blingfire

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 21.4 MB/s eta 0:00:00


In [ ]:
!pip install -q "cuvs-cu12==25.10.*" --extra-index-url=https://pypi.nvidia.com

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.0/840.0 MB 57.8 MB/s eta 0:00:00


### 캐글 데이터 준비

- 필요한 데이터셋 3개
  1. `kaggle-llm-science-exam`: 대회 훈련 데이터 (`train.csv`)
  2. `jjinho/wikipedia-20230701`: 위키피디아 parquet 파일 + 인덱스
  3. `mbanaei/wikipedia-2023-07-faiss-index`: 사전 구축된 FAISS 인덱스

In [ ]:
import os
from google.colab import files

!pip -q install kaggle

print("Please upload your kaggle.json file. You can generate it from your Kaggle 'Account' page.")
files.upload()

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


Please upload your kaggle.json file. You can generate it from your Kaggle 'Account' page.


Saving kaggle.json to kaggle.json


In [ ]:
!kaggle competitions download -c kaggle-llm-science-exam
# !kaggle datasets download -d jjinho/wikipedia-20230701
# !kaggle datasets download -d mbanaei/wikipedia-2023-07-faiss-index

kaggle-llm-science-exam.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip -q -o kaggle-llm-science-exam.zip -d kaggle-llm-science-exam
# !unzip -q -o wikipedia-20230701.zip -d wikipedia-20230701
# !unzip -q -o wikipedia-2023-07-faiss-index.zip -d wikipedia-2023-07-faiss-index

!echo "=== kaggle-llm-science-exam ===" && ls kaggle-llm-science-exam/
!echo "=== wikipedia-20230701 ===" && ls wikipedia-20230701/ | head -5
!echo "=== wikipedia-2023-07-faiss-index ===" && ls wikipedia-2023-07-faiss-index/

=== kaggle-llm-science-exam ===
sample_submission.csv  test.csv  train.csv
=== wikipedia-20230701 ===
ls: cannot access 'wikipedia-20230701/': No such file or directory
=== wikipedia-2023-07-faiss-index ===
ls: cannot access 'wikipedia-2023-07-faiss-index/': No such file or directory


In [ ]:
import os
import gc
import cudf as pd
import numpy as np
import re
from tqdm.auto import tqdm
import blingfire as bf
from collections.abc import Iterable

import faiss
from faiss import write_index, read_index

from sentence_transformers import SentenceTransformer

import warnings
import cupy as cp
from cuvs.neighbors import cagra

warnings.filterwarnings("ignore")

MODEL = 'sentence-transformers/all-MiniLM-L6-v2'
DEVICE = 0
MAX_LENGTH = 384
BATCH_SIZE = 16
WIKI_PATH = "/content/wikipedia-20230701"
NUM_SENTENCES_INCLUDE = 3

### 텍스트 문장 분할 함수 정의

- 3개 함수로 구성된 텍스트 전처리 파이프라인
  - `process_documents()`: 전체 흐름 관리 (문서 -> 섹션 -> 문장)
  - `sectionize_documents()`: 문서를 섹션 단위로 분리
  - `sentencize()`: 개별 문장 분할, `filter_len` 이하 짧은 문장 제거
- cuDF 사용 시 `.to_arrow().to_pylist()`로 Python 리스트 변환

In [ ]:
def process_documents(documents: Iterable[str],
                      document_ids: Iterable,
                      split_sentences: bool = True,
                      filter_len: int = 3,
                      disable_progress_bar: bool = False) -> pd.DataFrame:
    """
    EMR에서 문서를 처리하는 주요 도우미 함수입니다.
    :param documents: 문자열인 문서를 포함하는 반복 가능 객체
    :param document_ids: 문서 고유 식별자를 포함하는 반복 가능 객체
    :param split_sentences: 섹션을 문장으로 더 분할할지 여부를 결정하는 플래그
    :param filter_len: 문장의 최소 문자 길이(그렇지 않으면 필터링)
    :param disable_progress_bar: tqdm 진행률 표시줄을 비활성화하는 플래그
    :return: `document_id`, `text`, `section`, `offset` 열을 포함하는 Pandas DataFrame
    """
    df = sectionize_documents(documents, document_ids, disable_progress_bar)

    if split_sentences:
        df = sentencize(df.text.to_arrow().to_pylist(),
                        df.document_id.to_arrow().to_pylist(),
                        df.offset.to_arrow().to_pylist(),
                        filter_len,
                        disable_progress_bar)
    return df


def sectionize_documents(documents: Iterable[str],
                         document_ids: Iterable,
                         disable_progress_bar: bool = False) -> pd.DataFrame:
    """
    이미징 보고서의 섹션을 가져오고 선택한 섹션만 반환합니다.
    :param documents: 문자열인 문서를 포함하는 반복 가능 객체
    :param document_ids: 문서 고유 식별자를 포함하는 반복 가능 객체
    :param disable_progress_bar: tqdm 진행률 표시줄을 비활성화하는 플래그
    :return: `document_id`, `text`, `offset` 열을 포함하는 Pandas DataFrame
    """
    processed_documents = []
    for document_id, document in tqdm(zip(document_ids, documents), total=len(documents), disable=disable_progress_bar):
        row = {}
        text, start, end = (document, 0, len(document))
        row['document_id'] = document_id
        row['text'] = text
        row['offset'] = (start, end)
        processed_documents.append(row)

    _df = pd.DataFrame(processed_documents)
    if _df.shape[0] > 0:
        return _df.sort_values(['document_id', 'offset']).reset_index(drop=True)
    else:
        return _df


def sentencize(documents: Iterable[str],
               document_ids: Iterable,
               offsets: Iterable[tuple[int, int]],
               filter_len: int = 3,
               disable_progress_bar: bool = False) -> pd.DataFrame:
    """
    문서를 문장으로 분할합니다.
    :param documents: 문자열인 문서를 포함하는 반복 가능 객체
    :param document_ids: 문서 고유 식별자를 포함하는 반복 가능 객체
    :param offsets: 시작 및 끝 인덱스의 반복 가능 튜플
    :param filter_len: 문장의 최소 문자 길이(그렇지 않으면 필터링)
    :return: `document_id`, `text`, `section`, `offset` 열을 포함하는 Pandas DataFrame
    """
    document_sentences = []
    for document, document_id, offset in tqdm(zip(documents, document_ids, offsets), total=len(documents), disable=disable_progress_bar):
        try:
            _, sentence_offsets = bf.text_to_sentences_and_offsets(document)
            for o in sentence_offsets:
                if o[1] - o[0] > filter_len:
                    sentence = document[o[0]:o[1]]
                    abs_offsets = (o[0] + offset[0], o[1] + offset[0])
                    row = {}
                    row['document_id'] = document_id
                    row['text'] = sentence
                    row['offset'] = abs_offsets
                    document_sentences.append(row)
        except:
            continue

    return pd.DataFrame(document_sentences)

### ② 데이터 로딩 (위키피디아, 훈련 데이터)

- 위키피디아 파일 목록(`wiki_files`): 검색 대상 문서 파일 리스트
- 훈련 데이터(`trn`): 과학 5지선다 문제 (prompt + A~E 선택지 + answer)
- `SentenceTransformer` 모델 초기화
  - `device='cuda'`: GPU에서 임베딩 연산
  - `.half()`: FP16 변환으로 메모리 절약 및 속도 향상

In [ ]:
wiki_files = os.listdir(WIKI_PATH)

trn = pd.read_csv("/content/kaggle-llm-science-exam/train.csv")

model = SentenceTransformer(MODEL, device='cuda')
model.max_seq_length = MAX_LENGTH
model = model.half()

### ③ 프롬프트(prompt) 임베딩 생성 및 관련 문서 검색

- FAISS 인덱스 로드 -> 위키피디아 전체에서 관련 문서 Top-3 검색
- `normalize_embeddings=True`: L2 정규화 -> 코사인 유사도 기반 검색
- 검색 완료 후 인덱스/임베딩 메모리 해제 (`del` + `gc.collect`)
  - 이후 단계에서 GPU 메모리를 확보하기 위함

In [ ]:
sentence_index = read_index("/content/wikipedia-2023-07-faiss-index/wikipedia_202307.index")

prompt_embeddings = model.encode(
    trn.prompt.to_pandas().values,
    batch_size=BATCH_SIZE,
    device=DEVICE,
    show_progress_bar=True,
    convert_to_tensor=True,
    normalize_embeddings=True
).half()
prompt_embeddings = prompt_embeddings.detach().cpu().numpy()
_ = gc.collect()

search_score, search_index = sentence_index.search(prompt_embeddings, 3)

del sentence_index
del prompt_embeddings
_ = gc.collect()

### ④ 검색된 문서의 텍스트 로딩

- `wiki_2023_index.parquet`에서 문서 ID -> 파일 경로 매핑
- FAISS 검색 결과(`search_index`)의 문서 ID로 해당 parquet 파일 특정
- 파일별로 필요한 문서만 선별 로드 -> 전체 위키 로드 불필요
- 중복 제거 후 파일/ID 순 정렬

In [ ]:
df = pd.read_parquet(f"{WIKI_PATH}/wiki_2023_index.parquet", columns=['id', 'file'])

wikipedia_file_data = []
for i, (scr, idx) in tqdm(enumerate(zip(search_score, search_index)), total=len(search_score)):
    scr_idx = idx
    _df = df.loc[scr_idx].copy()
    _df['prompt_id'] = i
    wikipedia_file_data.append(_df)

wikipedia_file_data = pd.concat(wikipedia_file_data).reset_index(drop=True)
wikipedia_file_data = wikipedia_file_data[['id', 'prompt_id', 'file']].drop_duplicates().sort_values(['file', 'id']).reset_index(drop=True)

del df
_ = gc.collect()

In [ ]:
wiki_text_data = []
for file in tqdm(wikipedia_file_data.file.unique().to_arrow().to_pylist(), total=len(wikipedia_file_data.file.unique())):
    _id = [str(i) for i in wikipedia_file_data[wikipedia_file_data['file'] == file]['id'].to_arrow().to_pylist()]
    _df = pd.read_parquet(f"{WIKI_PATH}/{file}", columns=['id', 'text'])
    _df = _df[_df['id'].isin(_id)]
    wiki_text_data.append(_df)
    _ = gc.collect()

wiki_text_data = pd.concat(wiki_text_data).drop_duplicates().reset_index(drop=True)
_ = gc.collect()

### ⑤ 문서를 문장 단위로 분할하고 임베딩 생성

- 검색된 위키피디아 문서를 `process_documents()`로 문장 단위 분할
- 분할된 각 문장을 `SentenceTransformer`로 임베딩
  - 이 임베딩이 이후 CAGRA 인덱스 구축의 입력이 됨
- `normalize_embeddings=True`: 코사인 유사도 기반 비교를 위한 L2 정규화

In [ ]:
processed_wiki_text_data = process_documents(
    wiki_text_data.text.to_arrow().to_pylist(),
    wiki_text_data.id.to_arrow().to_pylist()
)

wiki_data_embeddings = model.encode(
    processed_wiki_text_data.text.to_arrow().to_pylist(),
    batch_size=BATCH_SIZE,
    device=DEVICE,
    show_progress_bar=True,
    convert_to_tensor=True,
    normalize_embeddings=True
).half()
wiki_data_embeddings = wiki_data_embeddings.detach().cpu().numpy()
_ = gc.collect()

### ⑥ 질문과 선택지 결합 후 임베딩 생성

- 질문(prompt) + 5개 선택지(A~E)를 하나의 문자열로 결합
  - `prompt_answer_stem` = 질문 + 모든 선택지 텍스트
- 결합 텍스트를 임베딩 -> 질문만 임베딩하는 것보다 더 풍부한 의미 벡터
- 이 벡터로 위키피디아 문장 중 가장 관련성 높은 컨텍스트를 검색

In [ ]:
trn['answer_all'] = trn['A'].str.cat([trn['B'], trn['C'], trn['D'], trn['E']], sep=" ")
trn['prompt_answer_stem'] = trn['prompt'] + " " + trn['answer_all']

question_embeddings = model.encode(
    trn['prompt_answer_stem'].to_arrow().to_pylist(),
    batch_size=BATCH_SIZE,
    device=DEVICE,
    show_progress_bar=True,
    convert_to_tensor=True,
    normalize_embeddings=True
).half()
question_embeddings = question_embeddings.detach().cpu().numpy()

### ⑦ GPU 기반 유사도 검색을 통한 컨텍스트 추출 (cuVS CAGRA)

- **FAISS 대신 cuVS CAGRA**를 사용한 GPU 가속 ANN 검색
- CAGRA 핵심 API
  - `cagra.IndexParams()`: 그래프 기반 인덱스 빌드 파라미터
  - `cagra.build(params, dataset)`: GPU에서 인덱스 구축
  - `cagra.SearchParams(max_queries, itopk_size)`: 검색 파라미터 설정
  - `cagra.search(params, index, query, k)`: GPU에서 Top-k 유사 벡터 검색
- 데이터 흐름: `numpy` -> `cupy`(`cp.asarray`) -> CAGRA 입출력
- 필터링: `distance < 2`인 후보만 컨텍스트로 추출
- 결과 포맷: `Question + Choices + Context` (RAG 입력 형태)

In [ ]:
prompt_contexts = []
contexts = []

index_params = cagra.IndexParams()
wiki_data_embeddings = cp.asarray(wiki_data_embeddings, dtype=cp.float32)
wiki_index = cagra.build(index_params, wiki_data_embeddings)

trn_pd = trn.to_pandas()

for r in trn_pd.itertuples():
    prompt_context = ""
    prompt_id = r.id
    context = ""

    if prompt_id >= len(question_embeddings):
        print(f"prompt_id {prompt_id} 가 question_embeddings 범위를 벗어납니다. 건너뜁니다.")
        continue

    query_vector = cp.asarray(question_embeddings[prompt_id], dtype=cp.float32).reshape(1, -1)

    search_params = cagra.SearchParams(max_queries=100, itopk_size=64)

    try:
        distances, indices = cagra.search(search_params, wiki_index, query_vector, NUM_SENTENCES_INCLUDE)
    except Exception as e:
        print(f"검색 실행 오류 (prompt_id {prompt_id}): {e}")
        continue

    try:
        prompt_text = trn_pd.at[prompt_id, 'prompt']
        choice_a = trn_pd.at[prompt_id, 'A']
        choice_b = trn_pd.at[prompt_id, 'B']
        choice_c = trn_pd.at[prompt_id, 'C']
        choice_d = trn_pd.at[prompt_id, 'D']
        choice_e = trn_pd.at[prompt_id, 'E']
    except Exception as e:
        print(f"질문 혹은 선택지 접근 실패 (prompt_id {prompt_id}): {e}")
        continue

    prompt_context += f"Question: {prompt_text}\n"
    prompt_context += "Choices:\n"
    prompt_context += f"(A) {choice_a}\n"
    prompt_context += f"(B) {choice_b}\n"
    prompt_context += f"(C) {choice_c}\n"
    prompt_context += f"(D) {choice_d}\n"
    prompt_context += f"(E) {choice_e}\n"

    if indices.shape[0] > 0:
        prompt_context += "Context:\n"
        distances_cpu = cp.asnumpy(distances)
        indices_cpu = cp.asnumpy(indices)

        for candidate_idx, candidate_distance in zip(indices_cpu[0], distances_cpu[0]):
            if candidate_distance < 2:
                if candidate_idx < processed_wiki_text_data.shape[0]:
                    candidate_text = processed_wiki_text_data['text'].iloc[candidate_idx]
                    context += "[*] " + candidate_text + "\n"
                else:
                    print(f"prompt_id {prompt_id}: 후보 인덱스 {candidate_idx} 가 processed_wiki_text_data 범위를 벗어났습니다.")

        prompt_context += context

    contexts.append(context)
    prompt_contexts.append(prompt_context)

### ⑧ 컨텍스트를 훈련 데이터에 추가하고 저장

- 검색된 컨텍스트를 `trn['context']` 컬럼에 추가
- `train_context.csv`로 저장 -> 이후 LLM 학습/추론에 활용

In [ ]:
trn['context'] = contexts
trn.to_csv("./train_context.csv", index=False)

### ⑨ 결과 확인

- 상위 10개 질문의 RAG 입력 형식 출력 (Question + Choices + Context)

In [ ]:
for i, p in enumerate(prompt_contexts[:10]):
    print(f"Question {i}")
    print(p)
    print()